In [1]:
%matplotlib qt

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.feature import match_template
from skimage.feature import peak_local_max

In [2]:
# Cargamos la imagen a procesar
img1 = cv.imread('C:/Users/c679529/Documents/GitHub/Computer_Vision1/Material_TPs/TP3/images/coca_logo_1.png')
img2 = cv.imread('C:/Users/c679529/Documents/GitHub/Computer_Vision1/Material_TPs/TP3/images/coca_logo_2.png')
img3 = cv.imread('C:/Users/c679529/Documents/GitHub/Computer_Vision1/Material_TPs/TP3/images/coca_retro_1.png')
img4 = cv.imread('C:/Users/c679529/Documents/GitHub/Computer_Vision1/Material_TPs/TP3/images/coca_retro_2.png')
img5 = cv.imread('C:/Users/c679529/Documents/GitHub/Computer_Vision1/Material_TPs/TP3/images/COCA-COLA-LOGO.jpg')
img6 = cv.imread('C:/Users/c679529/Documents/GitHub/Computer_Vision1/Material_TPs/TP3/images/logo_1.png')
template = cv.imread('C:/Users/c679529/Documents/GitHub/Computer_Vision1/Material_TPs/TP3/template/pattern.png',0)
imgz=[img1,img2,img3,img4,img5,img6]
for img in imgz:
    
    img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    

    #plt.figure()
    #plt.subplot(1,2,1)
    #plt.imshow(template, cmap='gray')
    #plt.subplot(1,2,2)
    #plt.imshow(img_rgb)



In [3]:
#Encapsulo todo el codigo para hacer un template matching 

def Template_Matching (img):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_blurred = cv.GaussianBlur(img_gray, (5, 5), 0)
    image_edges = cv.Canny(img_blurred, 350, 400)

    image_height, image_width = img.shape[:2]
    # Aplico Canny al template 
    template_edges = cv.Canny(template, 50, 150)

    #Defino un rango de procentajes que se van a aplicar  tanto a Height como Width 
    percentage_range = range(10, 400,10)  # Ajusto rango y step de movimiento 

    # Defino los posibles metodos a evaluar para encontrar el maximo de encuentro entre el template y la imagen 
    methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
                'cv.TM_CCORR_NORMED']

    # Inicializo las variables donde ire guardando lo resultados 
    best_method = ''
    best_result = None

    # Performo template matching para cada metodo y cada porcentaje 
    for method_str in methods:
        method = eval(method_str) 
        match_results = []

        # Performo template matching a cada resize de template 
        for percentage in percentage_range:
            try:
                # Callculo nuevas dimensiones segun porcentaje 
                original_height, original_width = template_edges.shape[:2]
                new_height = int(original_height * (percentage / 100))
                new_width = int(original_width * (percentage / 100))

                if new_height <= image_height and new_width <= image_width:
                    # Resize del template
                    resized_template = cv.resize(template_edges, (new_width, new_height))

                    # Performo template matching cno el respectivo metodo 
                    result = cv.matchTemplate(image_edges, resized_template, method)

                    # encuentro el lugar donde se encuentra el maximo de match 
                    _, max_val, _, max_loc = cv.minMaxLoc(result)

                    # guardo resultados 
                    match_results.append((max_val, max_loc, percentage))
            except cv.error:
                # este except esta puesto para evitar el caso en el cual el resize del template hace que sea mayor que la imagen ocasionando un error
                continue

        print(f"Method: {method_str}, Match results: {match_results}")

        # Agarro el resultado con mayor correlacion para el metodo en cuestion
        current_best_result = max(match_results, key=lambda x: x[0])

        print(f"Current best result: {current_best_result}")

        # Actualizo el best_method en caso de que mejore el previo 
        if best_result is None or current_best_result[0] > best_result[0]:
            best_result = current_best_result
            best_method = method_str

    # obtengo la informacion del mejor resultado 
    max_val, max_loc, best_percentage = best_result

    # Calculo dimensiones dependiendo 
    original_height, original_width = template_edges.shape[:2]
    new_height = int(original_height * (best_percentage / 100))
    new_width = int(original_width * (best_percentage / 100))

    # Resize del template para visualizacion 
    resized_template = cv.resize(template_edges, (new_width, new_height))

    # dibujo un rectangulo donde se encuentra la deteccion 
    h, w = resized_template.shape
    top_left = (max_loc[0], max_loc[1])
    bottom_right = (
        int(max_loc[0] + w),
        int(max_loc[1] + h),
    )
    cv.rectangle(img, top_left, bottom_right, 255, 2)

    
    text_size = int(0.2 * w)  # % del ancho
    text = f"Score: {max_val:.2f}"
    text_position = (top_left[0], top_left[1] - 10)

    # Calculate text size to use as the background rectangle size
    (_, text_height), _ = cv.getTextSize(text, cv.FONT_HERSHEY_SIMPLEX, text_size / 100, 2)


    # Draw a filled rectangle with additional height as the background for the text
    cv.rectangle(img, (text_position[0], text_position[1] - text_height - 10),
                (text_position[0] + w, text_position[1]), (0, 0, 0), -1)

    # Draw the text on top of the filled rectangle
    cv.putText(img, text, text_position, cv.FONT_HERSHEY_SIMPLEX, text_size / 100, (255, 255, 255), 2)

    

    # Muestro resultados 
    plt.figure()
    plt.subplot(121), plt.imshow(result, cmap='gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(img, cmap='gray')
    plt.title(f'Detected Logo using {best_method} at {best_percentage}% Resize')
    plt.show()


In [4]:
for img in imgz:
    Template_Matching(img)

Method: cv.TM_CCOEFF, Match results: [(611664.0, (80, 219), 10), (1457952.875, (68, 202), 20), (3004398.5, (35, 204), 30), (5382211.0, (34, 201), 40), (6553149.5, (5, 193), 50)]
Current best result: (6553149.5, (5, 193), 50)
Method: cv.TM_CCOEFF_NORMED, Match results: [(0.25429537892341614, (61, 19), 10), (0.11174221336841583, (61, 80), 20), (0.11107244342565536, (35, 204), 30), (0.12891872227191925, (34, 201), 40), (0.12269050627946854, (5, 193), 50)]
Current best result: (0.25429537892341614, (61, 19), 10)
Method: cv.TM_CCORR, Match results: [(1229865.0, (80, 219), 10), (3511349.5, (74, 223), 20), (6298754.0, (35, 204), 30), (9927404.0, (34, 201), 40), (11249070.0, (5, 193), 50)]
Current best result: (11249070.0, (5, 193), 50)
Method: cv.TM_CCORR_NORMED, Match results: [(0.29468315839767456, (80, 219), 10), (0.17976228892803192, (68, 202), 20), (0.20742085576057434, (35, 204), 30), (0.2144421488046646, (34, 201), 40), (0.19320479035377502, (5, 193), 50)]
Current best result: (0.29468

In [5]:
#PUNTO B - MULTI MATCHING 
img = cv.imread('C:/Users/c679529/Documents/GitHub/aprMaqI_CEIA/vision_computadora_I/Material_TPs/TP3/images/coca_multi.png')

img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.imread('C:/Users/c679529/Documents/GitHub/aprMaqI_CEIA/vision_computadora_I/Material_TPs/TP3/template/pattern.png',0)

plt.figure()
plt.subplot(1,2,1)
plt.imshow(template, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_rgb)



In [6]:
def template_matching_multi (img):
    # Aplico canny para deteccion de bordes 
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    image_edges = cv.Canny(img_gray, 50, 150)
    image_height, image_width = img.shape[:2]

    # Aplico canny al template 
    template_edges = cv.Canny(template, 50, 150)

    # Defino rango de valores de porcentaje y su respectivo step que afectaran el size del template 
    height_percentage_range = range(20,50, 2)
    width_percentage_range = range(20, 50, 2)

    # Defino el metodo 
    methods = ['cv.TM_CCORR_NORMED']

    # Inicializo variables para guardar resultados 
    best_method = []
    best_result = None

    # Performo template matching para el modelo 
    for method_str in methods:
        method = eval(method_str) 
        match_results = []
        results = []

        # Performo template matching para cada caso de  height and width
        for height_percentage in height_percentage_range:
            for width_percentage in width_percentage_range:
                try:
                    # Calculo las nuevas dimensiones para el template segun los %
                    original_height, original_width = template_edges.shape[:2]
                    new_height = int(original_height * (height_percentage / 100))
                    new_width = int(original_width * (width_percentage / 100))

                    if new_height <= image_height and new_width <= image_width:
                        # Resize del template
                        resized_template = cv.resize(template_edges, (new_width, new_height))

                        # Performo template matching
                        result = cv.matchTemplate(image_edges, resized_template, method)
                        results.append(result)

                        # Busco el lugar donde esta el mejor resultado
                        _, max_val, _, max_loc = cv.minMaxLoc(result)

                        # Guardo resultado 
                        match_results.append((max_val, max_loc, height_percentage, width_percentage))
                except cv.error:
                    # Except para tratar el caso en el que el resize de template hace que el size de la imagen sea menor ,ocasionando un error. 
                    continue

        print(f"Method: {method_str}, Match results: {match_results}")

        # Agarro el mejor resultado y su respectiva informacion 
        current_best_result = max(match_results, key=lambda x: x[0])

        
        print(f"Current best result: {current_best_result}")

        # Actualizo el best_method en caso de que mejore el previo
        if best_result is None or current_best_result[0] > best_result[0]:
            best_result = current_best_result
            best_method = method_str


    # OBtengo la info del mejor resultado
    max_val, max_loc, best_height_percentage, best_width_percentage = best_result

    # Calculo las nuevas dimensiones en las que se encontro el mejor resultado 
    original_height, original_width = template_edges.shape[:2]
    new_height = int(original_height * (best_height_percentage / 100))
    new_width = int(original_width * (best_width_percentage / 100))

    # Resize del template para mostrar resultados 
    resized_template = cv.resize(template_edges, (new_width, new_height))

    # Dibujo rectangulo en la zona encontrada
    h, w = resized_template.shape
    top_left = (max_loc[0], max_loc[1])
    bottom_right = (
        int(max_loc[0] + w),
        int(max_loc[1] + h),
    )
    best_methods = [best_method]
    for method_str in best_methods:
        method = eval(method_str)
        result = cv.matchTemplate(image_edges, resized_template, method)
    fig, ax = plt.subplots(figsize=(3, 3),dpi=300)
    ax.axis('off')
    ax.imshow(img_gray, cmap='gray')
    template_width, template_height = resized_template.shape
    for x, y in peak_local_max(result, threshold_abs=0.25):
        rect = plt.Rectangle((y, x), template_height, template_width, color='y', fc='none')
        plt.gca().add_patch(rect)
        #mostramos el score en cada encuentro 
        score_at_location = result[x, y]
        plt.text(y, x, f'{score_at_location:.2f}', color='r', fontsize=5)

In [7]:
template_matching_multi(img)

Method: cv.TM_CCORR_NORMED, Match results: [(0.24811187386512756, (372, 147), 20, 20), (0.2542068660259247, (564, 427), 20, 22), (0.23002737760543823, (463, 426), 20, 24), (0.23654182255268097, (406, 389), 20, 26), (0.23067426681518555, (200, 153), 20, 28), (0.2365770936012268, (67, 425), 20, 30), (0.23979708552360535, (15, 148), 20, 32), (0.2228643298149109, (391, 427), 20, 34), (0.22410713136196136, (542, 65), 20, 36), (0.22431284189224243, (399, 431), 20, 38), (0.22057676315307617, (253, 433), 20, 40), (0.21675404906272888, (170, 434), 20, 42), (0.21460634469985962, (523, 386), 20, 44), (0.21070703864097595, (381, 430), 20, 46), (0.21573975682258606, (164, 433), 20, 48), (0.24083127081394196, (202, 149), 22, 20), (0.26545390486717224, (26, 147), 22, 22), (0.2638867497444153, (403, 429), 22, 24), (0.24416974186897278, (407, 386), 22, 26), (0.24304796755313873, (483, 424), 22, 28), (0.24919895827770233, (263, 437), 22, 30), (0.23904509842395782, (532, 428), 22, 32), (0.236058071255683